<a href="https://colab.research.google.com/github/KEkerete/FadePredictionUsingTensoflow/blob/main/Switching_GRU_Stacked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Modifications: Mon Aug 28 00:59:37 2023   @author: keker
Modifications: Sun Aug 20 01:15:39 2023   @author: kdellubuntu
Created:       Thu Jun 27 20:20:04 2019   @author: ke0015
"""

#%%

# import sys
import warnings
import time
import csv
import numpy as np
import pandas as pd
from sys import platform
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# import matplotlib.pyplot as plt
# from matplotlib.pyplot import figure

def debug_signal_handler(signal, frame):  #pause execution with ctl+c, resume with c
    import pdb
    pdb.set_trace()

import signal
signal.signal(signal.SIGINT, debug_signal_handler)

#%% Define constants
min_switching_threshold, max_switching_threshold = 6, 14
training_pctge = 0.8
predictAheadMins = 5
maxMCNdB = 19.57
window_sizes = [5, 10, 15]
startRow, endRow = 0, 3000000

# Define file paths based on platform
if platform == 'win32':
    BASE_PATH = r'E:\UniBradford\Python\Scripts'
else:
    BASE_PATH = '/home/kdellubuntu/Python/researchData'

#%%
# =============================================================================
# def preprocess_data(scaler, sequence_length, df):
#     scaled_data = scaler.fit_transform(df)
#
#     X = [scaled_data[i:i + sequence_length] for i in range(len(scaled_data) - sequence_length)]
#     y = [scaled_data[i + sequence_length] for i in range(len(scaled_data) - sequence_length)]
#
#     X, y = np.array(X), np.array(y)
#
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
#     return X_train, X_test, y_train, y_test
#
# def train_and_evaluate(X_train, X_test, y_train, y_test):
#     # model = Sequential([
#     #     Bidirectional(GRU(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True)),
#     #     Dropout(0.2),
#     #     GRU(32, return_sequences=False),
#     #     Dense(16, activation='relu'),
#     #     Dense(1)
#     # ])
#
#     model = Sequential([
#         Bidirectional(GRU(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True)),
#         Dropout(0.2),
#         GRU(64, return_sequences=False),
#         Dense(32, activation='relu'),
#         Dense(1)
#     ])
#
#     # model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
#     model.compile(optimizer=Adam(learning_rate=0.0005), loss='mse')
#
#     # early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
#     # lr_scheduler = ReduceLROnPlateau(factor=0.2, patience=3, min_lr=1e-6)
#     early_stopping = EarlyStopping(patience=15, restore_best_weights=True)
#     lr_scheduler = ReduceLROnPlateau(factor=0.2, patience=5, min_lr=1e-6)
#
#     # Train the model
#     # history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.1, callbacks=[early_stopping, lr_scheduler], verbose=0)
#     history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping, lr_scheduler])
#
#     # Make predictions
#     train_predictions = model.predict(X_train)
#     test_predictions = model.predict(X_test)
#
#     # Inverse transform predictions for visualization
#     # train_predictions_inv = scaler.inverse_transform(train_predictions)
#     # test_predictions_inv = scaler.inverse_transform(test_predictions)
#
#     # Calculate RMSE
#     # train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions_inv))
#     # test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions_inv))
#
#     print(f"Window Size: {window_size}, Scaler: {scaler.__class__.__name__}")
#     print(f"Train RMSE: {train_rmse:.4f}")
#     print(f"Test RMSE: {test_rmse:.4f}")
#     print("-----------------------")
#
#     #     # # Visualization
#     #     # plt.figure(figsize=(12, 6))
#     #     # plt.plot(df.index[sequence_length:], df['Edin'][sequence_length:], label='Original Data')
#     #     # plt.plot(df.index[sequence_length:sequence_length + len(train_predictions_inv)], train_predictions_inv, label='Train Predictions', linestyle='dashed')
#     #     # plt.plot(df.index[sequence_length + len(train_predictions_inv):], test_predictions_inv, label='Test Predictions', linestyle='dashed')
#     #     # plt.xlabel('Time (secs)')
#     #     # plt.ylabel('Fade (dB)')
#     #     # plt.title('Time Series Prediction using Stacked GRU with Attention')
#     #     # plt.legend()
#     #     # plt.grid(which='major')
#     #     # plt.show()
#
#     # def plot_predictions(data, train_preds, test_preds):
#     plt.figure(figsize=(12, 6))
#     plt.plot(data, label='Original Data', color='blue')
#     plt.plot(np.arange(window_size, len(train_preds) + window_size), train_preds, label='Train Predictions', color='orange')
#     plt.plot(np.arange(len(train_preds) + window_size, len(train_preds) + len(test_preds) + window_size), test_preds, label='Test Predictions', color='green')
#     plt.xlabel('Time' (secs))
#     plt.ylabel('Fade Scaled Value (dB)')
#     plt.title('Original Data vs. Predicted Values')
#     plt.legend()
#     plt.show()
#
#
#
#
#     return train_predictions, test_predictions
#
#
# def GRU_Stacked(data):
#     df = pd.DataFrame(data)
#     window_sizes = [5, 10, 15]
#     # scalers = [StandardScaler(), MinMaxScaler()]
#     scaler = MinMaxScaler()
#     # results = []
#
#     for window_size in window_sizes:
#         # for scaler in scalers:
#         X_train, X_test, y_train, y_test = preprocess_data(scaler, window_size, df)
#         train_preds, test_preds = train_and_evaluate(X_train, X_test, y_train, y_test)
#
#         # scaler_name = scaler.__class__.__name__
#         # results.append((window_size, scaler_name, train_preds, test_preds))
#
#
#
# # Assuming 'original_data' is your original data in scaled form
# # original_data = scaled_data[window_size:]
# # plot_predictions(data, train_predictions, test_predictions)
#
#     return train_preds, test_preds  #results
# =============================================================================
#%%
def GRU_Stacked(data):

    # near perfect!

    # Load dataset
    # data = pd.read_csv('/home/kdellubuntu/researchData/TestDataChilEdin.csv')
    df = pd.DataFrame(data)   #, index=False, header=False)   # df = data

    # Experiment with different window sizes
    window_sizes = [5, 10, 15]

    for window_size in window_sizes:
        # Scaling the data with different scalers
        scalers = [StandardScaler(), MinMaxScaler()]

        for scaler in scalers:
            # Prepare data for training
            sequence_length = window_size
            X, y = [], []

            scaled_data = scaler.fit_transform(df)

            for i in range(len(scaled_data) - sequence_length):
                X.append(scaled_data[i:i + sequence_length])
                y.append(scaled_data[i + sequence_length])
            X, y = np.array(X), np.array(y)

            # Split data into training and testing sets
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

            # Model architecture
            model = Sequential([
                Bidirectional(GRU(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True)),
                Dropout(0.2),
                GRU(64, return_sequences=False),
                Dense(32, activation='relu'),
                Dense(1)
            ])

            model.compile(optimizer=Adam(learning_rate=0.0005), loss='mse')

            # Early Stopping and Learning Rate Scheduler
            early_stopping = EarlyStopping(patience=15, restore_best_weights=True)
            lr_scheduler = ReduceLROnPlateau(factor=0.2, patience=5, min_lr=1e-6)

            # Train the model
            history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping, lr_scheduler], verbose=0)

            # Make predictions
            train_predictions = model.predict(X_train)
            test_predictions = model.predict(X_test)

            # Inverse transform predictions for visualization
            train_predictions_inv = scaler.inverse_transform(train_predictions)
            test_predictions_inv = scaler.inverse_transform(test_predictions)

            # Calculate RMSE
            # train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions_inv))
            test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions_inv))

            # print(f"Window Size: {window_size}, Scaler: {scaler.__class__.__name__}")
            # print(f"Train RMSE: {train_rmse:.4f}")
            # print(f"Test RMSE: {test_rmse:.4f}")
            # print("-----------------------")

    # # Visualization
    # plt.figure(figsize=(12, 6))
    # plt.plot(data, label='Original Data', color='blue')
    # # plt.plot(df.index[sequence_length:], df['Edin'][sequence_length:], label='Original Data')
    # plt.plot(df.index[sequence_length:sequence_length + len(train_predictions_inv)], train_predictions_inv, label='Train Predictions', linestyle='dashed')
    # plt.plot(df.index[sequence_length + len(train_predictions_inv):], test_predictions_inv, label='Test Predictions', linestyle='dashed')
    # plt.xlabel('Time (secs)')
    # plt.ylabel('Fade (dB)')
    # plt.title('Time Series Prediction using Stacked GRU with Attention')
    # plt.legend()
    # plt.grid(which='major')
    # plt.show()

    # input('Press any key!')

    return train_predictions_inv, test_predictions_inv, test_rmse


#%%
# def main():
start_time = time.time()   #tic

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# Load data
series = read_csv(f'/home/Switching/Data/ChilboltonChiltonFadeData.csv')
mod_cod = read_csv(f'/home/Switching/Data/ModCod.csv')

total_atten_data = series.values

currentSiteCol = 0
currentFadeLevelCol = 1
currentSpecEffCol = 2
wSwitchData = tempSwitchData = [[0, 0, 0.0, 0, '00000000-00000', 0]]
specEffData, switchData = [], []
predDone = False

#%%  Correlation between data
# fadeData = series[['Chilbolton','Chilton']]
# correlation = fadeData.corr(method='pearson')
# print(correlation)
# plt.scatter(series.loc[:, 'Chilbolton'], series.loc[:, 'Chilton'])
# plt.show()

#%% Switching
if total_atten_data[0][1] < total_atten_data[0][2]:  # Initial starting site
    rxSite = 1  # Chilbolton
    altSite = 2
else:
    rxSite = 2  # Chilton
    altSite = 1

predictAheadData = predictAheadMins * 60    # seconds
trainData = int(predictAheadData * training_pctge / (1 - training_pctge))

#%% Loop for dB  minSwitchingThreshold, switchingThreshold
for switchingThreshold in range(min_switching_threshold, max_switching_threshold, 2):
    sameSiteCount, total_spec_eff, specEffCount = 0, 0, 0

    zeroCol = np.zeros((len(total_atten_data), 1))
    tData = np.hstack((total_atten_data, zeroCol, zeroCol, zeroCol))
    total_atten_data = tData
    del tData

    currentSiteCol += 3
    currentFadeLevelCol += 3
    currentSpecEffCol += 3

    for k in range(startRow, endRow):  #len(total_atten_data)):
        # snrdB = maxMCNdB - total_atten_data[k][rxSite] + 0.5      # snr
        # specEffIdx = mod_cod[(mod_cod['SNRdB-upper'] >= snrdB) & (mod_cod['SNRdB-lower'] < snrdB)].values
        # specEff = specEffIdx[0][4]
        # total_spec_eff += specEff             # total spectral efficiency
        sameSiteCount += 1

        if k > trainData:
            if total_atten_data[k][rxSite] > switchingThreshold:
                siteData = total_atten_data[k - trainData : k + predictAheadData,:]
                siteData = siteData[:,rxSite]

                # quit() ############

                # # YPred, cErr = pr.RandForest(siteData)
                # # YPred, trngErr, valErr = pr.GRU_Stacked(siteData)
                Y_train_pred, Y_test_pred, testRMSE = GRU_Stacked(siteData)
                predDone = True
                if np.amax(Y_test_pred) > total_atten_data[k][rxSite]:   # check if max(predictedFade) > currentFadeLevel

                    altSite = (rxSite % 2) + 1      # Alternate site

                if total_atten_data[k][altSite] < total_atten_data[k][rxSite]:   # check if fade(otherSite) < fade(currentSite)
                    siteData = total_atten_data[k - trainData : k + predictAheadData,:]
                    siteData = siteData[:,altSite]

                    # # YPred, cErr = pr.RandForest(siteData)
                    # # YPred, trngErr, valErr = pr.GRU_Stacked(siteData)
                    Y_train_pred, Y_test_pred, testRMSE  = GRU_Stacked(siteData)
                    predDone = True
                    # if np.amax(Y_test_pred) > total_atten_data[k][rxSite]:    # check if max(predictedFade) > currentFadeLevel in the altSite
                    if np.amax(Y_test_pred) < total_atten_data[k][rxSite]:    # check if max(predictedFade) < currentFadeLevel in the rxSite
                        if sameSiteCount >= 60: # must sustain for more than a minute
                            tempSwitchData.append([switchingThreshold, rxSite, total_atten_data[k-1][rxSite], sameSiteCount, total_atten_data[k][0], k])
                            wSwitchData = np.vstack((wSwitchData, tempSwitchData))
                            rxSite = altSite   # Switch to the alternate site
                            sameSiteCount = 0  # reset
                            tempSwitchData = [[0, 0, 0.0, 0, '00000000-00000', 0]]

                            #
        if predDone:  # if prediction was done
            for i in range(k, k + len(Y_test_pred)):
                # do not check the next len(Y_test_pred) samples
                # update the next len(Y_test_pred) samples with original data
                rxFade = total_atten_data[i][rxSite]
                total_atten_data[i, currentSiteCol] = rxSite
                total_atten_data[i, currentFadeLevelCol] = rxFade
                snrdB = maxMCNdB - total_atten_data[i][rxSite] + 0.5      # snr
                specEffIdx = mod_cod[(mod_cod['SNRdB-upper'] >= snrdB) & (mod_cod['SNRdB-lower'] < snrdB)].values
                specEff = specEffIdx[0][4]
                total_atten_data[i, currentSpecEffCol] = specEff
                specEffCount += 1
                # total_spec_eff += specEff             # total spectral efficiency

            k = min(len(total_atten_data), k + len(Y_test_pred))  # move k pointer
            predDone = False

        else:
            rxFade = total_atten_data[k][rxSite]
            total_atten_data[k, currentSiteCol] = rxSite
            total_atten_data[k, currentFadeLevelCol] = rxFade
            # total_atten_data[k, currentSpecEffCol] = specEff


        if k % 10000 == 0:
            if rxSite == 1:
                siteLoc = "Chilbolton"
            else:
                siteLoc = "Chilton"

            # print("\nDateTime={}, rxSite={}, Thresh(dB)={:,}/{:,} [{:,}/{:,}]\n".format(total_atten_data[k][0].strip(), siteLoc, switchingThreshold, maxSwitchingThreshold, k, len(total_atten_data)))
            print("\nTime={}, rxSite={}, Thresh(dB)={:,}/{:,} [{:,}/{:,}]\n".format(total_atten_data[k][0], siteLoc, switchingThreshold, max_switching_threshold, k, len(total_atten_data)))


    aveSpecEff = total_spec_eff / max(specEffCount, 1)
    # len(total_atten_data)
    print("\n\t Average Spectral Efficiency (FER) = {:5.2f}\n".format(aveSpecEff))

    if len(wSwitchData) > 1:
        switchData = wSwitchData[wSwitchData[:, 4] != '00000000-00000']

    # input("Press any key to continue!")

    #%% Save results

    newFadeData = total_atten_data[startRow:endRow,:]  #df.iloc[startRow:endRow]

    specEffData.append([switchingThreshold, aveSpecEff])

    if platform == 'win32':
        SwitchDataFile = r'E:\UniBradford\Python\Scripts\Results\SwDataFilePred#'+'{:,}'.format(switchingThreshold)+'mins.csv'
        specEffDataFile = r'E:\UniBradford\Python\Scripts\Results\SpEffDataFilePred.csv'
        newFadeFile = r'E:\UniBradford\Python\Scripts\Results\FadeFile.csv'
    elif platform == 'linux' or platform == 'linux2':
        SwitchDataFile = '/home/Switching/Results/SwDataFilePred#'+'{:,}'.format(switchingThreshold)+'mins.csv'
        specEffDataFile = '/home/Switching/Results/SpEffDataFilePred.csv'
        newFadeFile = '/home/Switching/Results/FadeFile.csv'

    with open(SwitchDataFile, 'w', newline='') as outfile:
        csv.writer(outfile).writerows(switchData)

    try:
        with open(specEffDataFile, 'w', newline='') as outfile:
            csv.writer(outfile).writerow(specEffData)
    except:
        with open(specEffDataFile, 'a', newline='') as outfile:
            csv.writer(outfile).writerow(specEffData)

    with open(newFadeFile, 'a',  newline='') as outfile:
        csv.writer(outfile).writerows(newFadeData)



# #%%########################
# # input("Press any key to cintinue!")

# color_map = plt.get_cmap('tab10')  # You can choose other color maps as well

# figure(num=None, figsize=(10, 6), dpi=80, facecolor='w', edgecolor='k')
# plt.title("Rain Fades and Switching for Chilbolton and Chilton", fontsize=14)  #%%s current_date));
# # plt.plot([row[1] for row in total_atten_data], "b", markersize=10, label="rxFade")  #
# # plt.plot([row[0] for row in total_atten_data], "r", markersize=10, label="Site (1 = Chilbolton, 2 = Edinburgh)",linewidth=2.0)  # rxSite
# plt.plot(total_atten_data[:,1], "b", markersize=10, label="Chilbolton")  #
# plt.plot(total_atten_data[:,2], "r", markersize=10, label="Chilton")  #

# j = 3
# for idx in range(min_switching_threshold, max_switching_threshold, 2):
#     color = color_map(idx)
#     plt.plot(2*j + total_atten_data[:,j], color=color, markersize=10, label=f'{idx} dB')  #
#     j=j+3

# plt.xlim(0, 2000000)  # len(total_atten_data))
# plt.legend(loc="best")
# plt.xlabel("Time (seconds)")
# plt.ylabel("Rain fade (dB)")
# plt.grid(which='major')
# plt.show()

elapsed_time = time.time() - start_time   #toc
print('\n\n ============================================ \n\n')
print("elapsedTime : {:9.9f} s".format(elapsed_time))
print('\n ..-Fin!-..\n')

# if __name__ == "__main__":
#     main()



Time=0.0, rxSite=Chilton, Thresh(dB)=6/14 [0/7,347,893]


Time=10000.0, rxSite=Chilton, Thresh(dB)=6/14 [10,000/7,347,893]


Time=20000.0, rxSite=Chilton, Thresh(dB)=6/14 [20,000/7,347,893]


Time=30000.0, rxSite=Chilton, Thresh(dB)=6/14 [30,000/7,347,893]


Time=40000.0, rxSite=Chilton, Thresh(dB)=6/14 [40,000/7,347,893]

10/10 [==============================] - 0s 5ms/step
